In [88]:
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import os
import math

In [89]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [90]:
images_open = os.listdir('eye_open') #Warning: in all images, BOTH eyes must be open
images_closed = os.listdir('eye_closed') #Warning: in all images, BOTH eyes must be closed

print(len(images_open), len(images_closed))

88 60


In [91]:
def calculate_roll_angle(landmarks):
    left_ear_x = landmarks[234].x
    left_ear_y = landmarks[234].y
    right_ear_x = landmarks[454].x
    right_ear_y = landmarks[454].y
    angle_rad = math.atan2(right_ear_y - left_ear_y, right_ear_x - left_ear_x)
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def compensate_head_roll(frame, results_mesh, W, H):
    roll_angle_deg = calculate_roll_angle(results_mesh.multi_face_landmarks[0].landmark)
    M = cv2.getRotationMatrix2D((W/2, H/2), roll_angle_deg, 1)
    frame = cv2.warpAffine(frame, M, (W, H))
    return frame

In [92]:
def update_mesh_points(image):
    try:
        frame = image.copy()
    except AttributeError:
        return None
    H, W, _ = frame.shape
    results_mesh_local = face_mesh.process(frame)
    if results_mesh_local.multi_face_landmarks:
        frame = compensate_head_roll(frame, results_mesh_local, W, H)
        results_mesh = face_mesh.process(frame)
        if results_mesh.multi_face_landmarks:
            mesh_points = np.array([np.multiply([p.x, p.y], [W, H]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
            for point in mesh_points:
                cv2.circle(frame, tuple(point), 1, (0, 255, 0), -1)
            return mesh_points
    return None

In [93]:
RIGHT_EYE = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
LEFT_EYE = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

def get_width_over_height(frame):
    mesh_points = update_mesh_points(frame)
    if mesh_points is not None:
        lex1 = mesh_points[33][0]
        lex2 = mesh_points[133][0]
        rex1 = mesh_points[362][0]
        rex2 = mesh_points[263][0]
        ley1 = mesh_points[159][1]
        ley2 = mesh_points[145][1]
        rey1 = mesh_points[386][1]
        rey2 = mesh_points[374][1]
        reason_left = abs(lex1-lex2)/abs(ley1-ley2)
        reason_right = abs(rex1-rex2)/abs(rey1-rey2)
        return (reason_left, reason_right)
    return None

In [94]:
results_open = []
results_closed = []

for image in images_open:
    frame = cv2.imread('eye_open/'+image)
    result = get_width_over_height(frame)
    if result:
        #print(result, image)
        results_open.append(result[0])
        results_open.append(result[1])

for image in images_closed:
    frame = cv2.imread('eye_closed/'+image)
    result = get_width_over_height(frame)
    if result:
        #print(result, image)
        results_closed.append(result[0])
        results_closed.append(result[1])

C:\Users\felip\AppData\Local\Temp\ipykernel_19372\2059583326.py:15: RuntimeWarning: divide by zero encountered in scalar divide
  reason_left = abs(lex1-lex2)/abs(ley1-ley2)
C:\Users\felip\AppData\Local\Temp\ipykernel_19372\2059583326.py:16: RuntimeWarning: divide by zero encountered in scalar divide
  reason_right = abs(rex1-rex2)/abs(rey1-rey2)


In [95]:
print(len(results_open), len(results_closed))

130 82


In [96]:
results_open = np.array(results_open)
results_closed = np.array(results_closed)

In [97]:
df = pd.DataFrame(columns=['width_over_height', 'label'])
for value in results_open:
    if value > 20:
        value = 20
    df = df.append({'width_over_height': value, 'label': 0}, ignore_index=True)
for value in results_closed:
    if value > 20:
        value = 20
    df = df.append({'width_over_height': value, 'label': 1}, ignore_index=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   width_over_height  212 non-null    float64
 1   label              212 non-null    float64
dtypes: float64(2)
memory usage: 3.4 KB


C:\Users\felip\AppData\Local\Temp\ipykernel_19372\445982613.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'width_over_height': value, 'label': 0}, ignore_index=True)
C:\Users\felip\AppData\Local\Temp\ipykernel_19372\445982613.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'width_over_height': value, 'label': 0}, ignore_index=True)
C:\Users\felip\AppData\Local\Temp\ipykernel_19372\445982613.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'width_over_height': value, 'label': 0}, ignore_index=True)
C:\Users\felip\AppData\Local\Temp\ipykernel_19372\445982613.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versio

In [98]:
df

,width_over_height,label
0,2.000000,0.0
1,2.000000,0.0
2,3.500000,0.0
3,2.333333,0.0
4,2.333333,0.0
...,...,...
207,20.000000,1.0
208,6.250000,1.0
209,20.000000,1.0
210,20.000000,1.0


In [99]:
df.to_csv('data.csv', index=False)